# Optimization Problem
In this tutorial, we will learn how to solve the following optimization problem using Gurobi solver.
\begin{align*}
& \max && x + y + z \\
& \textrm{subject to} && x + y = 1 \\
&&& x + 5 y + 2 z \le 10 \\
&&& 0 \le x \le 5 \\
&&& z \ge 2
\end{align*}

There are three components of above optimization problem:
### Decisions: $x, y, z$
### Objective function: $x + y + z$
### Constraints: $x + y = 1, x + 5 y + 2 z \le 10, 0 \le x \le 5, z \ge 2$


#### Write the above optimization problem as below in a notepad file and save it as 'model.lp'

```
Maximize
  x + y + z
Subject To
  c0: x + y = 1
  c1: x + 5 y + 2 z <= 10
Bounds
  0 <= x <= 5
  z >= 2
Generals
  x y z
End
```

To solve this model, we start a python session. We begin by importing Gurobi python package. 

In [106]:
import gurobipy as gp # import gurobipy package as gp
from gurobipy import GRB # Import class GRB from gurobi package.
import sys

Next step is to read the model. ```gp.read()``` reads the model from file. The argument of this function should be the location of ```.lp``` file. Note that the location uses forward slashes ```/``` not backward slashes ```\```. We assign the model read from this file to a variable named ```model```.

In [122]:
model = gp.read('D:/Down/model.lp')

Read LP format model from file D:/Down/model.lp
Reading time = 0.00 seconds
: 2 rows, 3 columns, 5 nonzeros


To switch off the verbose, we set ```model.Params.outputFlag``` to 0. If we want to see the details of the solution method, we set it equal to 1. Now it is time to solve the model. ```model.optimize()``` optimizes the model. 

In [108]:
model.Params.outputFlag = 0
model.optimize()

After we optimize, we need to find out whether the model is:

* Feasible and Gurobi has found the optimal solution ```GRB.OPTIMAL```
* Infeasible ```GRB.INFEASIBLE```
* Unbounded ```GRB.UNBOUNDED```
* Not sure if unbounded or infeasible ```GRB.INF_OR_UNBD``` 
  
This can be done by checking ```model.status``` and printing the appropriate status. 
  

In [109]:
if model.status == GRB.OPTIMAL:
    print('The model is feasible and we have found an optimal solution.')
elif model.status == GRB.INF_OR_UNBD:
    print('Model is infeasible or unbounded')
    sys.exit(0)
elif model.status == GRB.INFEASIBLE:
    print('Model is infeasible')
    sys.exit(0)
elif model.status == GRB.UNBOUNDED:
    print('Model is unbounded')
    sys.exit(0)
else:
    print('Optimization ended with status %d' % model.status)
    sys.exit(0)

The model is feasible and we have found an optimal solution.


The current model is optimal. Let us print the optimal objective value using ```model.objVal```.

In [110]:
print(model.objVal)

5.5


Now, let us check the optimal value of $x, y, z$. ```model.getVars()``` gives us the list of all the decision variables. ```k.x``` returns the value, whereas ```k.varName``` returns the name of the variable.

In [111]:
for k in model.getVars():
    print(k.varName ,' = ', k.x)

x  =  1.0
y  =  0.0
z  =  4.5


### How to get the dual variables or shadow prices?
This can be done by typing the constraint name and then use ```.pi```

In [112]:
for k in model.getConstrs():
    print('Shadow price of ', k, '=', k.pi)

Shadow price of  <gurobi.Constr c0> = 0.5
Shadow price of  <gurobi.Constr c1> = 0.5


### What if the model is infeasible. 

In [113]:
model = gp.read('D:/Down/model.lp')
model.Params.outPutFlag = 0
model.optimize()
if model.status == GRB.INFEASIBLE:
    print('Model is infeasible')
    sys.exit(0)
elif model.status == GRB.INF_OR_UNBD:
    print('Model is infeasible or unbounded')
    sys.exit(0)  

Read LP format model from file D:/Down/model.lp
Reading time = 0.00 seconds
: 2 rows, 3 columns, 5 nonzeros


Right now, the solver is not sure if the model is infeasible or unbounded. To clarify this, we set the ```model.Params.DualReductions``` to 0 and then re-optimize.

In [103]:
model.Params.DualReductions  = 0
model.optimize()
if model.status == GRB.INFEASIBLE:
    print('Model is infeasible')

Model is infeasible


To further explore the infeasibility, we can compute an Irreducible  inconsistent Subsystem (IIS) using ```model.computeIIS()```  and write it as a file. 

In [123]:
model.computeIIS() 
model.write("D:/Down/infeasible_model.lp")


IIS computed: 0 constraints and 2 bounds
IIS runtime: 0.00 seconds


### What if the model is unbounded
We can get the extreme ray using ```UnbdRay```

In [120]:
model.Params.DualReductions  = 0
model.Params.InfUnbdInfo = 1
model.Params.outPutFlag = 0
model.optimize()
for k in model.getVars():
    print(k.varName, ' = ', k.UnbdRay)

x  =  0.0
y  =  0.0
z  =  0.5
